# Mammalian3

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya
import numpy as np

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.Mammalian3)

class Mammalian3(pyagingModel):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        x_cpg = x[:, :-1707]  # number of species in lookup table
        x_species = x[:, -1707:]  # number of species in lookup table
        x = self.base_model(x_cpg)
        x = self.postprocess(x, x_species)
        return x

    def preprocess(self, x):
        return x

    def postprocess(self, x, x_species):
        """
        Converts output of to units of years.
        """
        indices = torch.argmax(x_species, dim=1)
        anage_array = self.postprocess_dependencies[0]
        anage_tensor = torch.tensor(anage_array, dtype=x.dtype, device=x.device)

        gestation_time = anage_tensor[indices, 0].unsqueeze(1)
        average_maturity_age = anage_tensor[indices, 1].unsqueeze(1)
        m_hat = 5 * (gestation_time / average_maturity_age) ** (0.38)

        # Create a mask for negative and non-negative values
        mask_negative = x < 0
        mask_non_neg

In [3]:
model = pya.models.Mammalian3()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'mammalian3'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'multi'
model.metadata["year"] = 2023
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Lu, A. T., et al. \"Universal DNA methylation age across mammalian tissues.\" Nature aging 3.9 (2023): 1144-1166."
model.metadata["doi"] = "https://doi.org/10.1038/s43587-023-00462-6"
model.metadata["research_only"] = None
model.metadata["notes"] = None

## Download clock dependencies

#### Download GitHub repository

In [5]:
github_url = "https://github.com/shorvath/MammalianMethylationConsortium.git"
github_folder_name = github_url.split('/')[-1].split('.')[0]
os.system(f"git clone {github_url}")

0

#### Download from R package

In [6]:
%%writefile download.r

options(repos = c(CRAN = "https://cloud.r-project.org/"))

myinput.list=readRDS('MammalianMethylationConsortium/UniversalPanMammalianClock/ClockParameters/mydata_GitHub.Rds')
anage=myinput.list[[3]]
anage=subset(anage,select=c(SpeciesLatinName,GestationTimeInYears, averagedMaturity.yrs,maxAge))
anage$HighmaxAge=1.3*anage$maxAge
anage$HighmaxAge[anage$SpeciesLatinName=='Homo sapiens']=anage$maxAge[anage$SpeciesLatinName=='Homo sapiens']
anage$HighmaxAge[anage$SpeciesLatinName=='Mus musculus']=anage$maxAge[anage$SpeciesLatinName=='Mus musculus']
write.csv(anage, "species_annotation.csv")

Writing download.r


In [7]:
os.system("Rscript download.r")

0

## Load features

#### From CSV file

In [8]:
df = pd.read_csv('MammalianMethylationConsortium/UniversalPanMammalianClock/ClockParameters/clock3.csv')
df['feature'] = df['var']
df['coefficient'] = df['beta_clock3']
cpg_features = df['feature'][1:].tolist()

anage_df = pd.read_csv('species_annotation.csv', index_col=0)
anage_df = anage_df[~anage_df['averagedMaturity.yrs'].isna()]
anage_df = anage_df[~anage_df['GestationTimeInYears'].isna()]
anage_df = anage_df.reset_index().drop('index', axis=1)
anage_df = anage_df.fillna(0)
species_features = anage_df['SpeciesLatinName'].tolist()

model.features = cpg_features + species_features

## Load weights into base model

In [9]:
weights = torch.tensor(df['coefficient'][1:].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'][0]])

#### Linear model

In [10]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [11]:
reference_list = np.array([0] * len(model.features))
reference_list[len(cpg_features) + np.where(anage_df.SpeciesLatinName == 'Homo sapiens')[0][0]] = 0.5
model.reference_values = reference_list

## Load preprocess and postprocess objects

In [12]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [13]:
model.postprocess_name = 'mammalian3'
anage_df = anage_df.loc[:, ['GestationTimeInYears','averagedMaturity.yrs', 'maxAge',	'HighmaxAge']]
anage_array = np.array(anage_df)
model.postprocess_dependencies = [anage_array]

## Check all clock parameters

In [14]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'Lu, A. T., et al. "Universal DNA methylation age across '
             'mammalian tissues." Nature aging 3.9 (2023): 1144-1166.',
 'clock_name': 'mammalian3',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.1038/s43587-023-00462-6',
 'notes': None,
 'research_only': None,
 'species': 'multi',
 'version': None,
 'year': 2023}
reference_values: array([0, 0, 0, ..., 0, 0, 0])
preprocess_name: None
preprocess_dependencies: None
postprocess_name: 'mammalian3'
postprocess_dependencies: [array([[2.19178082e-02, 2.49315068e+00, 3.60000000e+01, 4.68000000e+01],
       [1.64383562e-02, 3.00000000e+00, 1.15000000e+01, 1.49500000e+01],
       [3.01369863e-02, 4.50000000e+00, 1.50000000e+01, 1.95000000e+01],
       ...,
       [1.48575342e-01, 1.15317808e+00, 0.00000000e+00, 0.00000000e+00],
       [6.95616438e-02, 

## Basic test

In [15]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[ 9.0804],
        [ 0.2373],
        [-0.0573],
        [-0.1587],
        [25.9575],
        [-0.4438],
        [-0.0849],
        [-0.2192],
        [-0.0791],
        [ 0.0677]], dtype=torch.float64, grad_fn=<IndexPutBackward0>)

## Save torch model

In [16]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [17]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: species_annotation.csv


Deleted folder: MammalianMethylationConsortium
Deleted file: download.r
